In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np


# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
import pandas as pd
#import pyarrow.feather as feather

import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas



import matplotlib.pyplot as plt
import seaborn as sns

import importlib



import holoviews as hv


hv.extension("bokeh")

In [ ]:
# Get the path to the data

Datapath = Utils.get_data_path()

In [ ]:
# Find folders with "Learning or learning" in the name as a list

folders = [f for f in Datapath.glob("*FeedingState*")]

folders

In [ ]:
# Take the first 

exp = Ballpushing_utils.Experiment(folders[0])

In [ ]:
exp

In [ ]:
# For each fly, get the ball positions

ball_positions = []

for fly in exp.flies:
    data = fly.tracking_data.balltrack.objects[0].dataset
    
    data["y_relative"] = abs(data["y_centre"] - data["y_centre"].iloc[0])
    data['fly'] = fly.metadata.name
    
    ball_positions.append(data)
    
ball_positions = pd.concat(ball_positions)

ball_positions.head()

#ball_positions['fly'].unique()
    
    

In [ ]:
# Now get the average ball positions grouped by time

grouped = ball_positions.groupby('time')["y_relative"].mean()

# Make it a dataframe

grouped = grouped.reset_index()

# Add co
grouped["ci"] = (ball_positions.groupby('time')["y_relative"].sem()) * 1.96


grouped

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress

sns.set_style("white")
sns.set_style("ticks")

plt.rcParams.update({'font.size': 11})

# For each fly, get the ball positions
ball_positions = []

for fly in exp.flies:
    data = fly.tracking_data.balltrack.objects[0].dataset
    
    data["y_relative"] = abs(data["y_centre"] - data["y_centre"].iloc[0])
    data['fly'] = fly.metadata.name
    
    ball_positions.append(data)
    
ball_positions = pd.concat(ball_positions)

# Sample 20 flies from the pool of exp.flies
sampled_flies = np.random.choice(ball_positions['fly'].unique(), 20, replace=False)

# Now get the average ball positions grouped by time
grouped = ball_positions.groupby('time')["y_relative"].mean().reset_index()

# Plot the individual ball relative positions over time in thin gray lines for the sampled flies
plt.figure(figsize=(10, 6))
for fly in sampled_flies:
    sns.lineplot(x='time', y='y_relative', data=ball_positions[ball_positions['fly'] == fly], color='gray', alpha=0.3, linewidth=0.5)

# Plot the average ball positions over time in a thicker line
sns.lineplot(x='time', y='y_relative', data=grouped, label='Average ball position', color='blue', linewidth=2)

# Add a linear regression line to the plot and annotate the R2
# slope, intercept, r_value, p_value, std_err = linregress(grouped['time'], grouped['y_relative'])
# plt.plot(grouped['time'], slope * grouped['time'] + intercept, color='red', linestyle='--', alpha=0.5, label='Linear regression', linewidth=1)

# # Add the R2 to the plot
# plt.text(6500, 110, f"R2 = {r_value**2:.2f}", fontsize=12, color='red')

# Add title and labels
plt.title('Average ball position over time')
plt.xlabel('Time (s)')
plt.ylabel('Position relative to starting position (px)')
#plt.legend()
# Remove legend

plt.legend().remove()

plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Article_Affordance/average_ball_position_over_time_Simple.png")
plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Article_Affordance/average_ball_position_over_time_Simple.svg")
# Show the plot
plt.show()

In [ ]:
Dataset = Ballpushing_utils.Dataset(exp)

In [ ]:
Dataset.generate_dataset(success_cutoff=False)

In [ ]:
Dataset.data.columns

In [ ]:
# Plot the yball_relative as a function of time grouped by individual flies

# curves = hv.Curve(Dataset.data, kdims=["time"], vdims=["yball_relative","fly"]).groupby("fly").overlay()

# curves.opts(width=800, height=400)

In [ ]:
from holoviews import opts
from bokeh.palettes import viridis

Tries = 30

NbFlies = 7

for i in range(Tries):

    # Pick random flies and plot their yball_relative as a function of time
    flies = np.random.choice(Dataset.data.fly.unique(), NbFlies)

    # Generate a Viridis palette with adapted number of colors colors
    viridis_palette = viridis(NbFlies)

    # Create the curves and overlay them
    curves = (
        hv.Curve(
            Dataset.data[Dataset.data.fly.isin(flies)],
            kdims=["time"],
            vdims=["yball_relative", "fly"],
        )
        .groupby("fly")
        .overlay()
    )

    curves = curves.opts(
        opts.Curve(
            width=800,
            height=400,
            show_grid=True,
            show_legend=False,
            ylabel="ball position relative to start (px)",
            xlabel="time (s)",
            
        )
    )

    # Apply the viridis colormap
    curves = curves.opts(opts.Curve(color=hv.Cycle(values=list(viridis_palette))))

    hv.save(curves, f"/mnt/upramdya_files/DURRIEU_Matthias/DlsImgs/testcurves{i}.png")

# Plotting curve with event start positions

In [ ]:
exp = Ballpushing_utils.Experiment(folders[0])

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
Dataset = Ballpushing_utils.Dataset(exp)

In [ ]:
Dataset.data.head()

In [ ]:
SubsetFly = Dataset.data[Dataset.data["fly"] == Dataset.data["fly"].unique()[0]]

SubsetFly["interaction_event_onset"].unique()

In [ ]:
SubsetFly[SubsetFly["interaction_event_onset"].notna()]

In [ ]:
#Show rows where interaction_event_onset is not NaN

Dataset.data[Dataset.data["interaction_event_onset"].notna()]

In [ ]:
# Let's try to get the events for one fly

ex_fly = Dataset.data[Dataset.data['fly'] == Dataset.data['fly'].unique()[1]]

In [ ]:
ex_fly["interaction_event"]

In [ ]:
# Check non null values

ex_fly["interaction_event"].notnull().sum()

In [ ]:
events

In [ ]:
# Plot y fly

plt.plot(ex_fly["time"], ex_fly["y_fly_0"])

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots()

# Plot yball_relative over time
ax.plot(ex_fly["time"], ex_fly["distance_ball_0"], label="distance_ball_0", color="black")

# Find the beginning of each event
event_starts = (
    ex_fly.dropna(subset=["interaction_event"])
    .groupby("interaction_event")
    .first()
    .reset_index()
)

# Mark the beginning of each event with a red 'X'
ax.scatter(
    event_starts["time"],
    event_starts["distance_ball_0"],
    color="red",
    marker="x",
    s=100,
    label="Event Start",
)

# Add labels and legend
ax.set_xlabel("Time")
ax.set_ylabel("distance_ball_0")
ax.legend()

# Show the plot
plt.show()

In [ ]:
# Select the fly to highlight
highlight_fly_id = Dataset.data['fly'].unique()[1]  # Replace with the actual ID of the fly to highlight

plt.rcParams.update({'font.size': 22})

# Create a figure and axis with specific size

fig, ax = plt.subplots(figsize=(30, 20))

# Plot all other animals in shades of gray
other_flies = Dataset.data[Dataset.data["fly"] != highlight_fly_id]
sns.lineplot(
    data=other_flies,
    x="time",
    y="distance_ball_0",
    hue="fly",
    palette=sns.color_palette("husl", len(other_flies["fly"].unique())),
    # palette="gray",
    alpha=0.3,
    legend=False,
    ax=ax,
)

# Plot the highlighted animal in black
highlight_fly = Dataset.data[Dataset.data["fly"] == highlight_fly_id]
sns.lineplot(
    data=highlight_fly,
    x="time",
    y="distance_ball_0",
    color="black",
    label="distance_ball_0",
    ax=ax,
)

# Find the beginning of each event for the highlighted animal
event_starts = (
    highlight_fly.dropna(subset=["interaction_event_onset"])
    .groupby("interaction_event_onset")
    .first()
    .reset_index()
)

# Mark the beginning of each event with a red 'X'
ax.scatter(
    event_starts["time"],
    event_starts["distance_ball_0"],
    color="red",
    marker="x",
    s=100,
    label="Event Start",
)

# Add labels and remove legend
ax.set_xlabel("time (s)")
ax.set_ylabel("ball distance to start (px)")
ax.legend().remove()

plt.savefig(
    "/mnt/upramdya_files/DURRIEU_Matthias/Pictures/Affordance_paper/BallTrajectories_2.svg",
    format="svg",
)

plt.savefig(
    "/mnt/upramdya_files/DURRIEU_Matthias/Pictures/Affordance_paper/BallTrajectories_2.pdf",
    format="pdf",
)


# Show the plot
plt.show()

In [ ]:
# Select the fly to highlight
highlight_fly_id = Dataset.data['fly'].unique()[1]  # Replace with the actual ID of the fly to highlight

plt.rcParams.update({'font.size': 22})

# Create a figure and axis with specific size
fig, ax = plt.subplots(figsize=(30, 20))

# Plot all other animals in shades of gray
other_flies = Dataset.data[Dataset.data["fly"] != highlight_fly_id]
sns.lineplot(
    data=other_flies,
    x="time",
    y="distance_ball_0",
    hue="fly",
    palette=sns.color_palette("husl", len(other_flies["fly"].unique())),
    # palette="gray",
    alpha=0.3,
    legend=False,
    ax=ax,
)

# Get the highlighted fly data and sort by time
highlight_fly = Dataset.data[Dataset.data["fly"] == highlight_fly_id].sort_values("time")

# Create new columns to identify interaction segments
highlight_fly['is_interaction'] = ~highlight_fly["interaction_event"].isna()
highlight_fly['segment'] = (highlight_fly['is_interaction'] != highlight_fly['is_interaction'].shift()).cumsum()

# Plot each segment with appropriate color
legend_added = {'red': False, 'black': False}
for segment_id, segment in highlight_fly.groupby('segment'):
    color = 'red' if segment['is_interaction'].iloc[0] else 'black'
    
    # Only add label for the first occurrence of each color
    if not legend_added[color]:
        label = 'Interaction' if color == 'red' else 'No Interaction'
        legend_added[color] = True
    else:
        label = None
        
    ax.plot(segment["time"], segment["distance_ball_0"], color=color, label=label, linewidth=2)

# Find the beginning of each event for the highlighted animal
event_starts = (
    highlight_fly.dropna(subset=["interaction_event_onset"])
    .groupby("interaction_event_onset")
    .first()
    .reset_index()
)

# Mark the beginning of each event with a red 'X'
ax.scatter(
    event_starts["time"],
    event_starts["distance_ball_0"],
    color="red",
    marker="x",
    s=100,
    label="Contact onset",
)

# Add labels and legend
ax.set_xlabel("time (s)")
ax.set_ylabel("ball distance to start (px)")
ax.legend()

# Place legend top left

plt.legend(loc='upper left')

plt.savefig(
    "/mnt/upramdya_files/DURRIEU_Matthias/Pictures/Affordance_paper/BallTrajectories_2_Withinteractions.svg",
    format="svg",
)

plt.savefig(
    "/mnt/upramdya_files/DURRIEU_Matthias/Pictures/Affordance_paper/BallTrajectories_2_Withinteractions.pdf",
    format="pdf",
)

# Show the plot
plt.show()